In [ ]:
import pandas as pd

data = pd.read_csv("src/difference_in_difference_with_deep_learning/data/testdata.csv")

In [ ]:
import yaml

with open(
    "src/difference_in_difference_with_deep_learning/data_management/data_info.yaml",
) as file:
    data_info = yaml.safe_load(file)

In [ ]:
column_select = data_info["categorical_columns"]

In [ ]:
data = data[column_select]

In [ ]:
column1 = data_info["categorical_columns"][1]
columns2 = data_info["categorical_columns"][2]

In [ ]:
data_select_1 = data[data_info["categorical_columns"][1]]
data_select_2 = data[data_info["categorical_columns"][2]]

In [ ]:
data_select_1 = data[data_info["categorical_columns"][1]]
data_select_2 = data[data_info["categorical_columns"][2]]

In [ ]:
data.loc[:, "interaction"] = data_select_1 * data_select_2

In [ ]:
data

In [ ]:
data["interaction"] = data_select_1 * data_select_2

In [ ]:
use_df = data(columns=data_info["categorical_columns"])

In [ ]:
data["new"] = data["Reform"] * data["FQ"]